# 11. OCR 기술의 개요

### OCR = Text detection + Text recongnition
![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-6-L-00.max-800x600.png)

딥러닝 기반 OCR이 어떻게 이루어지는지 알아보자.   
OCR은 두 과정을 합친것인데, 이미지 속의 문자 영역을 검출하는 text detection, 검출된 영역의 문자를 인식하는 text recongnition으로 구분된다.

__text detection__   
일반적인 object detection task를 문자로 확장한 버전이라 생각할 수 있다. 하지만 segmentation 기법도 동원되며, 문자가 가지는 독특한 특성까지 고려한 형태로 발전해오고 있다.

__text recognition__   
검출된 영역의 문자가 무엇인지 인식하는 과정이다.   
text detecion을 통해 검출된 문자 영역 속에는 MNIST 데이터처럼 문자 단위로 분리된 텍스트 이미지만 담겨있는게 아니다. 그래서 OCR 분야에서 사용하는 독특한 모델 구조에 대해 다루게 될 것이다.

# 11-2. Before Deep Learning

OCR(Optical Character Recognition)은 이미지 속에서 글자를 인식해내는 기술이다.   
자동차 번호판 자동인식, 신용카드 광학인식 등 우리 생활 속에 OCR 기술이 자연스럽게 녹아들어 있다.

OCR의 시작은 딥러닝의 대중화인 2010년 초반대로부터 80년이나 이전인 1928년 오스트리아의 포토 센서를 활용한 패턴 매칭을 통해 제작한 "reading machine", 1931년 이스라엘의 "statistical machine"이 특허를 냈는데, 이들이 최초의 OCR 장치로 여겨지고 있다.   
OCR의 상용화는 1980년대부터 본격적으로 등장했다. 

- [From Videos to URLs: A Multi-Browser Guide To Extract User’s Behavior with Optical Character Recognition
](https://arxiv.org/pdf/1811.06193.pdf)

해당 논문을 통해 딥러닝이 OCR에 활용되기 전 어떻게 이미지속 문자를 찾았는지 알 수 있다.

브라우저에서 동작하는 OCR을 이용하여 웹에서 유저의 행동을 관찰하는 방법을 제안한다. 온라인 마케팅, 광고 등에서 OCR 기술을 활용할 수 있다는 내용이다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GC-6-L-01.png)

OCR 엔진으로 Tesseract OCR을 사용하려 한 것을 확인할 수 있다.   
Tesseract OCR은 1985년 휴렛 패커드사에서 개발이 시작되어 2005년 오픈소스로 출시되었고, 2006년부터 구글의 후원을 받는 오픈소스 OCR 엔진이다. 최근에는 Tesseract OCR에도 LSTM을 적용하는 등 딥러닝 베이스 아키텍처로 발전하고 있다.

위 그림은 크게 5단계로 나뉘어 있는데, 위 3단계는 입력 이미지 추출과 전처리, 4번째는 OCR, 5번째는 OCR의 출력 텍스트 후처리로 구성된다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/ARCHIT1.max-800x600.png)

Tesseract OCR을 통한 4단계 과정을 정리한 그림이다.

`Adaptive Thresholding` 단계에서 입력 영상의 이진화를 통해 흑백으로 변환한다. `Connected Component Analysis` 단계에서 문자 영역을 검출한 후 `Find Lines and Words` 단계에서 라인, 단어 단위룰 추출한다. `Recognize word` 단계에서 단어 단위로 이미지를 text로 변환하기 위해 문자를 하나씩 인식하고 다시 결합하는 과정을 거친다.

딥러닝 기반 OCR도 위처럼 복잡할 수 있지만 기본적인 기능을 하기 위해 필요한 단계가 많다. 딥러닝의 적용을 통해 원하는 단위로 문자를 검출하고, 한 번에 인식하도록 아키텍처를 단순화하여 빠른 인식을 이뤄낼 수 있다. 심지어 검출과 인식을 동시에 하는 End-to-End OCR 모델도 연구되고 있다.

# 11-3. Text detection

위에서 확인한 Tesseract OCR도 text detection + recognition 흐름은 큰 차이가 없다.

딥러닝을 이용해 이미지에서 텍스트 위치를 찾는 text detection 방법에 대해 생각해보자.   
object detection, segmentation 기법들을 떠올릴 수 있는데, 이미지 속에서 물체와 문자는 조금 다른 특성을 갖고 있다. 문자는 몇 개가 모여 단어 혹은 문장을 이뤄내 이미지 내에서 문자를 검출할 때 검출하기 위한 __최소 단위__를 정해야 한다.

예를 들어 이미지 속에서 문장 단위로 위치를 찾거나 단어나 글자 단위로 위치를 찾게끔 정할 수 있다.   
문장, 단어 단위로 찾아낼 경우 엄청 긴 단어나 문장과 함께 짧은 길이도 찾아낼 수 있도록 해야한다.   
그래서 글자 단위로 인식하는 방법도 있는데 이는 글자를 놓치지 않고 찾아내는데 좋을 수 있지만, 글자를 다시 맥락에 맞게 묶어주는 과정을 거쳐야 한다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-6-L-02.max-800x600.png)

2017년에 발표된 [EAST: An Efficient and Accurate Scene Text Detector](https://arxiv.org/pdf/1704.03155v2.pdf) 논문에서 소개된 다양한 detection 기법을 정리한 것이다.

2017년 당시에는 text의 바운딩 박스를 구하는 방식이 주류를 이루었다.   
가로 방향(horizontal)으로만 텍스트 박스를 구하는 방식이나, 기울어지거나 세로 방향 등의 여러 multi oriented 텍스트 박스를 구하는 방식이 다양하게 소개되고 있다.   
(e)의 경우 전체 파이프라인의 길이가 짧고 간결해서 빠르고 정확한 text detection 성능을 보인다고 소개한다.

word, text단위의 detection이 모두 활용되고 있다.  
단어 단위의 탐지는 object detection의 regression 기반의 detection 방법이다.   
Anchor를 정의하고 단어의 유무와 bounding box의 크기를 추정해서 단어를 찾아낸다. 글자 단위의 방식은 bounding box regression을 사용하는 대신 글자 영역을 segmentation 하는 방식으로 접근한다. 두 가지 모두 활용한 방법도 있다.

이러한 대표적인 여러 가지 접근 방식을 알아보자.

### 1. Regression
[TextBoxes: A Fast Text Detector with a Single Deep Neural Network](https://arxiv.org/pdf/1611.06779.pdf)

![](https://d3s0tskafalll9.cloudfront.net/media/images/architecture_of_textboxes.max-800x600.png)

TextBoxes는 2017년 공개된 논문이다. 이미지에서 문자를 찾아내려는 방법에 다양한 기법들이 활용된다.

TextBoxes 이전에는 글자 단위로 인식한 후 결합하는 방식을 취했지만, 위 논문에서는 딥러닝 기반 detection을 이용해서 단어 단위로 인식한다. 네트워크의 기본 구조는 [SSD: single shot multibox detector](https://arxiv.org/pdf/1512.02325.pdf)를 활용했다. 이를 통해 빠르게 문자 영역을 탐지할 수 있다고 한다.

일반적으로 단어들은 가로로 길기 때문에 aspect ratio(종횡비)가 크다. 그래서 몇 가지 변형을 주게 되는데, 기존의 SSD에서는 regression을 위한 convolution layer에서 3x3 kernel이 긴 단어의 featrue를 활용하기 위해 1x5로 filter 사이즈를 정의한다.   
anchor box의 aspect ratio를 1,2,3,5,7로 만들고, 이에 vertical offset을 적용하여 세로 방향으로 촘촘한 단어의 배열에 대응할 수 있게 한다.

### 2. Segmentation
[PixelLink: Detecting Scene Text via Instance Segmentation](https://arxiv.org/pdf/1801.01315.pdf)

![](https://d3s0tskafalll9.cloudfront.net/media/images/segmentation_map.max-800x600.png)

semantic segmentation을 통해 이미지 내의 영역을 class로 분리하는 방법을 배웠다. 이 방법을 문자의 영역 찾기로 적용하면, 배경과 글자인 영역으로 분리할 수 있다.   
하지만 문자들은 매우 촘촘히 배열되어 있으므로, 글자 영역으로 찾아낸 뒤에 이를 분리해내는 작업이나 연결하는 작업을 추가적으로 더 한 뒤에 최소단위로 만들어줘야 한다.

PixelLink는 Text 영역을 찾아내는 segmentation과 함께, 글자가 어느 방향으로 연결되는지 같이 학습하여 text 영역의 분리 및 연결을 할 수 있는 정보를 추가적으로 활용한다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/architecture_pixellink.max-800x600.png)

PixelLink의 전체적인 구조는 U-Net과 유사하다. 하지만 output으로 총 9가지 정보를 얻는다.

그림의 녹색이 input, output을 의미하는데, output중 하나는 text/non-text prediction을 위한 class segmentation map으로 해당 영역이 text인지 아닌지 예측하는 2개의 커널을 가진다. 나머지 8가지는 글자의 pixel을 중심으로 인접한 8개의 pixel에 대한 연결 여부를 의미하는 16개의 커널로 이루어진 Link Prediction map이다.

conv 1x1, 2(16) 형태의 레이어가 U-Net 구조로 연결되어, 인접 pixel간 연결 구조가 지속적으로 유지되도록 하는 모델 구조이다.

이를 통해 인접한 pixel이 중심 pixel과 단어 단위로 연결된 pixel인지, 아니면 분리된 pixel인지 알 수 있으므로, 문자 영역이 단어 단위로 분리된 instance segementation이 가능해진다.

최근의 방법들을 알아보자.
### 3-1. CRAFT
[Character Region Awareness for Text Detection](https://arxiv.org/abs/1904.01941)

![](https://d3s0tskafalll9.cloudfront.net/media/images/craft_affinity_map.max-800x600.png)

CRAFT는 문자 단위로 문자의 위치를 찾아낸 뒤, 이를 연결하는 방식을 segmentation 기반으로 구현한 방법이다.   
문자의 영역을 boundary로 명확히 구분하지 않고, 가우시안 분포를 따르는 원형의 score map을 만들어서 배치시키는 방법으로 문자 영역을 학습한다.

문자 단위의 라벨을 가진 데이터셋이 많지 않아, 단어 단위의 정보만 있는 데이터셋에 대해 단어 영역에 Inference를 한 후, 얻어진 문자 단위의 위치를 다시 학습에 활용하는 weakly supervised learning을 활용했다.

### 3-2. Pyramid Mask Text Detector
[Pyramid Mask Text Detector](https://arxiv.org/pdf/1903.11800.pdf)

![](https://d3s0tskafalll9.cloudfront.net/media/images/PMTD.max-800x600.png)

PMTD는 Mask-RCNN의 구조를 활용하여 먼저 text 영역을 region proposal network로 찾아낸다.   
그 다음 box head에서 정확하게 regression 및 classification을 하고 mask head에서 instance의 segmentation을 하는 과정을 거친다.

PMTD는 Mask 정보가 부정확한 경우를 반영하기 위해 soft segmentation을 활용한다. 이전의 Mask-RCNN의 경우 단어 영역이 box head에 의해 빨간색으로 잡히면 위 이미지의 baseline처럼 boundary를 모두 text 영역으로 잡지만, PMTD는 단어의 사각형 배치 특성을 반영하여 피라미드 형태의 score map을 활용한다.   
그래서 pyramid 형상의 mask를 갖게 되어 pyramid mask text detector라는 이름을 붙이게 됐다.

# 11-4. Text recognition

### 1. Unsegmented Data
글자 이미지들은 문자의 순서대로 정보를 갖고 있다. "YOU" 라는 단어를 갖고 있는 이미지에서 "Y", "O", "U" 영역으로 분리가 가능하다.

분리에 드는 비용이 많거나 어려워 segmentation이 되어있지 않은 데이터를 Unsegmented data라고 한다. 이미지에서 문자를 읽어내는 OCR분야 뿐만 아니라 다른 분야들에서도 이런 데이터를 볼 수 있다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/unsegmented_data_copy.max-800x600.png)

예를들어 위 그림과 같이 annotation이 제대로 안 된 음성 데이터도 Unsegmented data의 한 종류이다. Waveform 파일에 대한 라벨이 "the sound of"로 달려있을 때, waveform 파일에서 어디까지가 "the", "sound", "of" 인지 경계가 명확하게 분할되어 있지 않은 경우를 생각할 수 있다.

### 2. CNN과 RNN의 만남 CRNN
[An End-to-End Trainable Neural Network for Image-based Sequence Recognition and Its Application to Scene Text Recognition](https://arxiv.org/pdf/1507.05717.pdf)

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/crnn.png)

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/crnn_structure.png)

Unsegmented Data들이 가진 중요한 특징 중 하나는 segment 되어 있지 않은 하위 데이터들끼리 시퀀스를 이루고 있다는 점이다.

그래서 연구자들이 CNN과 RNN을 같이 쓰는 방법을 생각해냈고, 이런 모델을 CRNN이라고 한다.

문자 이미지에서 정보를 추출하기 위해 Feature Extractor가 필요하다. Feature Extractor로 사용되는 CNN 기반의 VGG 또는 ResNet과 같은 네트워크를 통해 문자의 정보를 가진 feature를 얻어낼 수 있다.   
이렇게 얻은 feature를 Map-To-Sequence를 통해 sequence 형태의 feature로 변환 후, 다양한 길이의 input을 처리할 수 있는 RNN으로 넣는다.

RNN이 feature로부터 문자를 인식하기 위해 문자 영역처럼 넓은 정보가 필요하기 때문에, LSTM으로 구성한다.   
앞부터 뒤 정보까지 필요하기 때문에 이를 Bidirectional로 구성해서 bidrectional LSTM을 사용하고, 매 step마다 나오는 결과는 Transcription layer에서 문자로 변환된다.

### 3. CTC

CRNN에서 Step마다 FC layer의 logit을 softmax 함수에 넣어줌으로써 어떤 문자일 확률이 높은지 알 수 있다.

하지만 이 결과 그대로 문자로 변환하면 기대와 달리 다른 결과가 나온다. 왜냐하면 모델의 output은 24개의 글자로 이루어진 sequence이지만, 실제 결과는 이와 다르기 때문이다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/ctc.png)

예를들어 "hello"라는 이미지가 들어오면 이것의 Output이 "HHHEEELLLOOOO..."와 같이 24자의 sequence를 보게 된다.   
이 24자의 sequence를 실제 인식 결과로 바꾸기 위해 어떤 방법을 써야 할까?

CRNN에서는 Unsegmentation Data를 위해 [CTC(Connectionist Temporal Classification)](http://www.cs.toronto.edu/~graves/icml_2006.pdf)를 활용한다. CTC는 Unsegmented data와 같이 Input과 Output이 서로 다른 Length의 sequence를 가질 때, 이를 align 없이 활용하는 방법이다.

논문에서 언급하는 CTC의 핵심인 모델의 output에서 label sequence의 확률을 구할 수 있는 방법에 대해 알아보자.

"HHEEELLLOOOO.."를 "HELLO"로 만들기 위해 중복되는 단어들을 제거하면 "HELO"가 된다. "L"이 두 번 중복되어야 하는데 어떻게 구분할 수 있을까?

label encode에서 같은 문자를 구분하기 위한 blank를 중복된 라벨 사이에 구분하기 위해 넣어준다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/wbHRk.png)

Blank token을 '-'로 대신하여 Output을 만드는 decoder를 의미한다. decode 후에 중복을 제거하고, 인식할 문자가 아닌 값을 지워주면 "HELLO"라는 결과를 얻을 수 있다.

이렇게 인식된 text의 정확도를 실제 정답과 비교하기 위해 Edit distance라는 방법이 있다. 두 문자열 사이의 유사도를 판별하는 방법으로, 예측된 단어에서 삽입, 삭제, 변경을 통해 얼마나 적은 횟수의 편집으로 정답에 도달할 수 있는지 최소 거리를 측정한다.

### 4. TPS
[Robust Scene Text Recognition With Automatic Rectification](https://arxiv.org/pdf/1603.03915.pdf)

책이 아닌 거리의 글자를 읽을 때, OCR이 어려워지는 이유는 불규칙한 방향이나 휘어진 진행 방향 때문이다.

논문에서는 Thin Plate Spline(TPS) tranformation을 적용하여 입력 이미지를 단어 영역에 맞게 변형 시켜 인식이 잘 되도록 해준다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/thinplates-dy.max-800x600.png)

Thin plate spline은 control point를 정의하고 해당 point들이 특정 위치로 옮겨졌을 때, 축 방향의 변화를 interpolcation하여 모든 위치의 변화를 추정해낸다. 이를 통해 전체 이미지 pixel의 변화를 control point로 만들어낼 수 있다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-6-L-03.max-800x600.png)

논문에서 control point를 20개로 미리 정의한다. spatial transformer network를 통해 control point가 얼마나 움직여야 하는지 예측하는 네트워크를 그림과 같이 recognition model 앞단에 붙여 입력 이미지를 정방향으로 맞춰준다. TPS 연산은 미분 가능한 연산이기 때문에 이 모듈을 recognition model 앞단에 붙여 학습이 바로 가능하다.

# 11-5. Text recognition + Attention

Attention과 transformer가 딥러닝 분야에 큰 변화를 가져왔고, OCR도 예외는 아니다.

### 1. Attention sequence prediction
[Robust Scene Text Recognition With Automatic Rectification](https://arxiv.org/pdf/1603.03915.pdf)   
[Focusing Attention: Towards Accurate Text Recognition in Natural Images](https://arxiv.org/pdf/1709.02054.pdf)

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/rnn_decoder.png)

CTC를 활용한 CRNN의 경우, column에 따라 prediction된 label의 중복을 제거하며 원하는 형태의 label로 만들었다.

Attention 기반의 sequence prediction은 문장의 길이를 고정하고, 입력되는 feature에 대한 attention을 기반으로 해당 글자의 label을 prediction한다.   
RNN으로 charactor label을 뽑아낸다고 생각하면 되는데, 첫 번째 글자에서 입력 feature에 대한 Attention을 기반으로 label을 추정하고, 추정된 label을 다시 입력으로 사용하여 다음 글자를 추정하는 방식이다.

20 글자를 뽑겠다고 정하게 되면 "YOU" 같은 경우는 세 글자를 채우고 빈자리가 문제가 되는데, 이러한 경우는 미리 정해둔 Token을 사용한다.   
Token이 처음 사용되는 "start" token과 끝에 사용되는 "end" token이 있다. 필요에 따라 예외처리나 공백을 위한 token을 만들어서 사용하기도 한다.

[What Is Wrong With Scene Text Recognition Model Comparisons? Dataset and Model Analysis](https://arxiv.org/pdf/1904.01906.pdf)

Attention 기반의 Recognition이 더욱 좋은 성능을 보이는 것은 위의 naver clova의 논문을 통해 확인할 수 있다. 이 논문에서는 CTC와 attention만이 아닌 TPS 등 recognition에서 쓰이는 다양한 모듈들을 비교 평가 한다.

### 2. Transformer
[A Simple and Robust Convolutional-Attention Network for Irregular Text Recognition](https://arxiv.org/pdf/1904.01375v1.pdf)   
[hulk89님의 논문리뷰](https://hulk89.github.io/machine%20learning/2019/05/15/A-Simple-and-Robust-Convolutional-Attention-Network-For-Irregular-Text-Recognition/)

![](https://d3s0tskafalll9.cloudfront.net/media/images/transformer_rec.max-800x600.png)


tranformer도 recognition 모델에 활용되기 시작한다. 다양한 논문에서 시도되고 있지만 "a simple and robust convolutional attention network for lrregular text recognition"이 대표적이다.

위 논문에서 lrregular text(불규칙한 텍스트)를 잘 인식하기 위해 2d space에 대한 attention을 활용했고, 문자를 인식하기 위해 transformer를 활용한다.

transformer는 query, key, value라는 개념을 통해 self-attention을 입력으로부터 만들어낸다.   
이를 통해 입력에서 중요한 feature에 대해 weight를 주게 된다. attention의 핵심은 decoder의 현재 포지션에서 중요한 encoder의 state에 가중치가 높게 매겨진다는 점이다.

attention이 시각화한 이미지를 눈으로 볼 수 있는데, decoder의 각 step에 따라 입력에 대한 visual attention이 시각화된 모습이다.